### Import Libraries

In [70]:
import importlib
import os
import json
import sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from models import gemini

importlib.reload(gemini)

<module 'models.gemini' from 'c:\\Users\\annak\\Documents\\Multimodal-Techniques-for-Video-Based-Violence-Detection\\models\\gemini.py'>

In [71]:
BATCH_SIZE = 10
GEMINI_OUTPUT_JSON_PATH = "../../data/results/gemini_transcripts.json"
GEMINI_METADATA_OUTPUT_JSON_PATH = "../../data/results/gemini_metadata.json"

### Get list with video paths

In [ ]:
# video_urls_path = "../../data/metadata/video_links_labels.csv"
# video_urls = pd.read_csv(video_urls_path)
# video_urls.head(3)

,id,download_url,binary_label,multilabel
0,A.Beautiful.Mind.2001__#00-01-45_00-02-50_label_A,https://huggingface.co/datasets/jherng/xd-viol...,0,0
1,A.Beautiful.Mind.2001__#00-03-00_00-04-05_label_A,https://huggingface.co/datasets/jherng/xd-viol...,0,0
2,A.Beautiful.Mind.2001__#00-04-20_00-05-35_label_A,https://huggingface.co/datasets/jherng/xd-viol...,0,0


In [ ]:
# video_urls_list = video_urls["download_url"].to_list()
# video_urls_list[2]

### Run Gemini

In [72]:
# Get prompt
prompt_path = "../../data/prompts/video_understanding_gemini.txt"
with open(prompt_path, "r", encoding="utf-8") as f:
    prompt = f.read()

In [73]:
# Initialize the instance
gemini_model = gemini.GeminiAsyncRequester(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="models/gemini-2.5-pro",
    prompt=prompt,
)

In [74]:
xd_violence = os.path.join(os.getenv("XD_VIOLENCE_PATH"), "abuse2.mp4")

In [75]:
await gemini_model.run_async(
    [xd_violence],
    GEMINI_OUTPUT_JSON_PATH,
    GEMINI_METADATA_OUTPUT_JSON_PATH,
    concurrency_limit=4,
    overwrite=False,
    append=True,
    use_url=False,
)